In [4]:
import xarray as xr
import numpy as np
from pathlib import Path

def check_nc_resolution(nc_path, var_name):
    ds = xr.open_dataset(nc_path)
    
    # Find lat/lon variable names (works even if they are 'lat'/'lon')
    lat_name = [name for name in ds.coords if 'lat' in name.lower()][0]
    lon_name = [name for name in ds.coords if 'lon' in name.lower()][0]
    
    lat = ds[lat_name].values
    lon = ds[lon_name].values
    
    # Handle descending latitudes
    lat_res = np.abs(np.mean(np.diff(lat)))
    lon_res = np.abs(np.mean(np.diff(lon)))
    
    print(f"File: {nc_path}")
    print(f"Variable: {var_name}")
    print(f"Latitude spacing: {lat_res:.6f} degrees")
    print(f"Longitude spacing: {lon_res:.6f} degrees")
    
    lat_res_m = lat_res * 111_000  # m per degree latitude
    lon_res_m = lon_res * 111_000 * np.cos(np.deg2rad(np.mean(lat)))
    
    print(f"Approx pixel size: {lat_res_m:.2f} m (N-S), {lon_res_m:.2f} m (E-W)")

# Example usage
era5_nc = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc")
check_nc_resolution(era5_nc, var_name='t2m')


File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc
Variable: t2m
Latitude spacing: 0.250000 degrees
Longitude spacing: 0.250000 degrees
Approx pixel size: 27750.00 m (N-S), 24032.20 m (E-W)


In [7]:
import rasterio
from pathlib import Path

def check_pixel_size_tif_precise(tif_path):
    with rasterio.open(tif_path) as src:
        transform = src.transform
        pixel_width = transform.a
        pixel_height = -transform.e  # make positive
        crs = src.crs
        
        print(f"File: {tif_path}")
        print(f"CRS: {crs}")
        print(f"Pixel size (width): {pixel_width:.8f} {'m' if crs.is_projected else 'degrees'}")
        print(f"Pixel size (height): {pixel_height:.8f} {'m' if crs.is_projected else 'degrees'}")

# Example usage
lulc_file = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif")
check_pixel_size_tif_precise(lulc_file)


File: C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif
CRS: EPSG:4326
Pixel size (width): 0.00416667 degrees
Pixel size (height): 0.00416667 degrees


In [9]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pathlib import Path

def reproject_resample_to_30m(input_path, output_path):
    # Step 1: Open input file
    with rasterio.open(input_path) as src:
        print(f"Original CRS: {src.crs}")
        print(f"Original resolution: {src.res}")

        # Step 2: Choose a projected CRS (UTM based on center of dataset)
        utm_crs = src.crs
        if utm_crs.is_geographic:
            # Auto-pick a UTM zone based on dataset center
            from pyproj import CRS
            lon, lat = src.xy(src.height // 2, src.width // 2)
            utm_crs = CRS.from_user_input(f"+proj=utm +zone={(int((lon + 180) / 6) % 60) + 1} +datum=WGS84 +units=m +no_defs")
            print(f"Target UTM CRS: {utm_crs}")

        # Step 3: Calculate transform for 30 m resolution
        transform, width, height = calculate_default_transform(
            src.crs, utm_crs, src.width, src.height, *src.bounds, resolution=30
        )

        # Step 4: Define output profile
        profile = src.profile.copy()
        profile.update({
            'crs': utm_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Step 5: Reproject + resample
        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=utm_crs,
                    resampling=Resampling.bilinear  # or nearest for categorical (like LULC)
                )

    print(f"Saved reprojected 30 m raster → {output_path}")

# Example usage:
input_file = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif")  # Change path
output_file = input_file.with_name(input_file.stem + "_30m.tif")
reproject_resample_to_30m(input_file, output_file)


Original CRS: EPSG:4326
Original resolution: (0.004166667, 0.004166667)
Target UTM CRS: +proj=utm +zone=44 +datum=WGS84 +units=m +no_defs +type=crs
Saved reprojected 30 m raster → C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif


In [12]:
#checking
import rasterio

file_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m.tif"  # change path

with rasterio.open(file_path) as src:
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")  # (pixel_width, pixel_height)


CRS: EPSG:32644
Resolution: (30.0, 30.0)


In [13]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pathlib import Path

def reproject_resample_to_30m(input_path, output_path):
    # Step 1: Open input file
    with rasterio.open(input_path) as src:
        print(f"Original CRS: {src.crs}")
        print(f"Original resolution: {src.res}")

        # Step 2: Choose a projected CRS (UTM based on center of dataset)
        utm_crs = src.crs
        if utm_crs.is_geographic:
            # Auto-pick a UTM zone based on dataset center
            from pyproj import CRS
            lon, lat = src.xy(src.height // 2, src.width // 2)
            utm_crs = CRS.from_user_input(f"+proj=utm +zone={(int((lon + 180) / 6) % 60) + 1} +datum=WGS84 +units=m +no_defs")
            print(f"Target UTM CRS: {utm_crs}")

        # Step 3: Calculate transform for 30 m resolution
        transform, width, height = calculate_default_transform(
            src.crs, utm_crs, src.width, src.height, *src.bounds, resolution=30
        )

        # Step 4: Define output profile
        profile = src.profile.copy()
        profile.update({
            'crs': utm_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Step 5: Reproject + resample
        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=utm_crs,
                    resampling=Resampling.bilinear  # or nearest for categorical (like LULC)
                )

    print(f"Saved reprojected 30 m raster → {output_path}")

# Example usage:
input_file = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2016_clipped.tif")  # Change path
output_file = input_file.with_name(input_file.stem + "_30m.tif")
reproject_resample_to_30m(input_file, output_file)


Original CRS: EPSG:4326
Original resolution: (0.004166667, 0.004166667)
Target UTM CRS: +proj=utm +zone=44 +datum=WGS84 +units=m +no_defs +type=crs
Saved reprojected 30 m raster → C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2016_clipped_30m.tif


In [14]:
#checking
import rasterio

file_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2016_clipped_30m.tif"  # change path

with rasterio.open(file_path) as src:
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")  # (pixel_width, pixel_height)

CRS: EPSG:32644
Resolution: (30.0, 30.0)


In [2]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

lulc_in = r"C:\Users\Ankit\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped_30m_filled_categorical.tif"
era5_ref = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif"
lulc_out = r"C:\Users\Ankit\Datasets_Forest_fire\LULC_2015_resampled_to_ERA5.tif"

with rasterio.open(era5_ref) as ref:
    ref_profile = ref.profile
    ref_transform = ref.transform
    ref_crs = ref.crs
    ref_height, ref_width = ref.height, ref.width

with rasterio.open(lulc_in) as src:
    data = src.read(1)
    dst = np.empty((ref_height, ref_width), dtype=data.dtype)

    reproject(
        source=data,
        destination=dst,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        resampling=Resampling.mode   # <-- majority vote for categorical
    )

    profile = ref_profile.copy()
    profile.update(count=1, dtype=dst.dtype)

    with rasterio.open(lulc_out, "w", **profile) as out_ds:
        out_ds.write(dst, 1)

print("✅ LULC resampled with majority (mode) →", lulc_out)


✅ LULC resampled with majority (mode) → C:\Users\Ankit\Datasets_Forest_fire\LULC_2015_resampled_to_ERA5.tif


In [5]:
import rasterio
import numpy as np

lulc_file = r"C:\Users\Ankit\Datasets_Forest_fire\LULC_2015_resampled_to_ERA5.tif"

with rasterio.open(lulc_file) as src:
    print("Size:", src.width, "x", src.height)
    print("Bands:", src.count)
    data = src.read(1)
    unique_vals = np.unique(data)
    print("Unique values:", unique_vals[:20])  # safe slicing


Size: 17 x 13
Bands: 1
Unique values: [0 1]


In [7]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling

# Input files
era5_ref = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif"
lulc_in  = r"C:\Users\Ankit\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif"
lulc_out = r"C:\Users\Ankit\Datasets_Forest_fire\LULC_2015_resampled_to_ERA5.tif"

# Open ERA5 to use as template
with rasterio.open(era5_ref) as ref:
    ref_profile = ref.profile
    ref_transform = ref.transform
    ref_crs = ref.crs
    ref_h, ref_w = ref.height, ref.width

# Open LULC and resample to ERA5 grid
with rasterio.open(lulc_in) as src:
    src_data = src.read(1)
    src_nodata = src.nodata if src.nodata is not None else 255  # assume 255 = nodata for LULC

    # Destination grid
    dst = np.full((ref_h, ref_w), src_nodata, dtype=src_data.dtype)

    # Resample with nearest neighbour (safe for categorical)
    reproject(
        source=src_data,
        destination=dst,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        src_nodata=src_nodata,
        dst_nodata=src_nodata,
        resampling=Resampling.nearest
    )

    # Save output aligned with ERA5
    profile = ref_profile.copy()
    profile.update(count=1, dtype=dst.dtype, nodata=src_nodata)
    with rasterio.open(lulc_out, "w", **profile) as dst_ds:
        dst_ds.write(dst, 1)

print("✅ LULC resampled to ERA5 grid:", lulc_out)

# Check result
with rasterio.open(lulc_out) as check:
    arr = check.read(1)
    print("Resampled size:", arr.shape)   # should be (13, 17)
    print("Unique values in resampled LULC:", np.unique(arr))


✅ LULC resampled to ERA5 grid: C:\Users\Ankit\Datasets_Forest_fire\LULC_2015_resampled_to_ERA5.tif
Resampled size: (13, 17)
Unique values in resampled LULC: [  1   2  11  12  15  16  21  22  31  32 255]
